# Comparing NYC Mayoral Voting Patterns with Fare Payment Pattern

As the results of the 2025 NYC Mayoral Election were tallied, [several](https://www.nytimes.com/2025/11/05/opinion/mamdani-mayor.html) [articles] and [other] [people] noticed that precints who primarily use transit generally supported Zohran Mamdani, while those that principally drive generally supported Andrew Cuomo. At the same time, [several] [other] [sources] mention a shfiting political landscape away from one dominated by (transit-dependent) Manhattan.

An analysis I did last year of OMNY versus MetroCard use showed higher OMNY use was higher closer to Manhattan (and uniquely high in Williamsburg). I wanted to see how this distirbution lines up with support for Mamdani and Cuomo.

## 1. Dependencies

In [6]:
!pip install sodapy
!pip install geopandas
!pip install folium matplotlib mapclassify
!pip install geodatasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 882.2/882.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [folium]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [geodatasets]


In [7]:
import pandas as pd
from sodapy import Socrata
import geopandas as gpd
import geodatasets
from shapely.geometry import Point
import requests
import matplotlib.pyplot as plt
import matplotlib
import folium
import mapclassify
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression
from datetime import datetime

## 2. Importing Voting Data

## 3. Importing MTA Subway Data
MTA Data used in this project is from the Subway Hourly Ridership (beginning February 2022) at https://data.ny.gov/Transportation/MTA-Subway-Hourly-Ridership-Beginning-February-202/wujg-7c2s

In [ ]:
nys_client = Socrata("data.ny.gov", None)

In [ ]:
#personal_client = Socrata("data.cityofnewyork.us", None)

In [ ]:
def get_station_list(client):
    results = client.get("5f5g-n3cz")
    results_df = pd.DataFrame.from_records(results)
    station_list = results_df['complex_id'].values.tolist()
    return station_list


We will use OMNY and MetroCard data from October of 2025.

In [ ]:
def get_data(client, batch_size=100000, max_retries=3, start_date="2025-10-01", end_date="2025-10-30"):
    """retrieve data for all stations, with a given start and end date
        Note: retries should at most be n where 10^n is <= batch_size"""
    station_list = get_station_list(client)

    results = []
    failed_stations = []
    for station in station_list:
        retry_count = 0
        success = False
        this_station_result = []
        
        where_string = (f"station_complex_id = '{str(station)}' "
            f"AND transit_timestamp >= '{start_date}' "
            f"AND transit_timestamp <= '{end_date}'"
            )

        while (not success) and (retry_count < max_retries):
            try:
                this_station_result = client.get("wujg-7c2s", order="transit_timestamp DESC", where=where_string, limit=batch_size)
                success = True
            except Exception as e:
                retry_count += 1
                if retry_count == max_retries:
                    failed_stations += [station]

        this_df = pd.DataFrame(this_station_result)
        results += [this_df]
    
    results_df = pd.concat(results, ignore_index=True)

    return results_df, failed_stations
